In [66]:
BEST_OF = [1]

In [67]:
import pandas as pd

PROCESSED_FILES_FOLDER = '../data/processed'
OUTPUT_PATH = f'{PROCESSED_FILES_FOLDER}/full_matches_train_data.csv'

## Definindo as colunas que serão usadas para treinar o modelo
team_map_results_columns_to_add  = ["PlusMinus", "Adr", "Kast", "Rating"]
team_map_results_columns_to_drop  = []

map_results = [f"map{map_num}Team{team_num}Side{side}Player{player_num}{attribute}"
                for map_num in BEST_OF 
                for team_num in range(1, 3) 
                for player_num in range(1, 6)
                for attribute in team_map_results_columns_to_add
                for side in ["Both", "CounterTerrorist", "Terrorist"]]

kast_columns = [f"map{map_num}Team{team_num}Side{side}Player{player_num}{attribute}"
                for map_num in BEST_OF 
                for team_num in range(1, 3) 
                for player_num in range(1, 6)
                for attribute in ["Kast"]
                for side in ["Both", "CounterTerrorist", "Terrorist"]]

adr_columns = [f"map{map_num}Team{team_num}Side{side}Player{player_num}{attribute}"
                for map_num in BEST_OF 
                for team_num in range(1, 3) 
                for player_num in range(1, 6)
                for attribute in ["Adr"]
                for side in ["Both", "CounterTerrorist", "Terrorist"]]

### Listando as colunas que serão usadas das partidas, como: Jogadores, Mortes, Assistências etc.
matches_train_data_columns = ['eventId', 'matchId', 'mapBestOf'] + map_results

print(f"Reading data from {PROCESSED_FILES_FOLDER}/matches_results.csv and {PROCESSED_FILES_FOLDER}/matches.csv")

matches_results_train_data = pd.read_csv('../data/raw/matches_results.csv')
matches_train_data = pd.read_csv('../data/raw/matches.csv')

matches_train_data = matches_train_data[matches_train_data_columns]

matches_results_train_data_columns = ['eventId', 'matchId', 'TeamOneScore', 'TeamTwoScore', 'teamOneWon', 'teamTwoWon']
matches_results_train_data = matches_results_train_data[[col for col in matches_results_train_data.columns if any(s in col for s in matches_results_train_data_columns)]]

## Combinando os dados gerais das partidas com os detalhes das partidas
full_matches_train_data = pd.merge(matches_results_train_data, matches_train_data, on='matchId', how='inner')

full_matches_train_data = full_matches_train_data[full_matches_train_data['mapBestOf'].isin(BEST_OF)]

## Data Wrangling
full_matches_train_data.fillna(0, inplace=True)
full_matches_train_data.replace("Not Available", 0, inplace=True)
full_matches_train_data.drop(['eventId_y', 'eventId_x', 'matchId'], axis=1, inplace=True)

full_matches_train_data.drop(team_map_results_columns_to_drop, axis=1, inplace=True)
full_matches_train_data.drop(['mapBestOf'], axis=1, inplace=True)

## arredondando os valores de Kast e Adr
for column in kast_columns:
    full_matches_train_data[column] = full_matches_train_data[column].apply(lambda x: round(float(x.strip('%')) / 100, 2) if str(x) != '0' else 0)
for column in adr_columns:
    full_matches_train_data[column] = full_matches_train_data[column].apply(lambda x: round(float(x) / 100, 2))

## Convertendo as colunas para float
# for column in full_matches_train_data.columns:
#     full_matches_train_data[column] = full_matches_train_data[column].astype(float)

full_matches_train_data.to_csv(OUTPUT_PATH, index=False)

display(full_matches_train_data.head())
display(full_matches_train_data.shape)

print(f"Data saved to {OUTPUT_PATH}")

Reading data from ../data/processed/matches_results.csv and ../data/processed/matches.csv


/var/folders/10/t4cd3lc515j88jjkvk8hvrbw0000gn/T/ipykernel_7721/2589967565.py:37: DtypeWarning: Columns (35,36,37,41,42,43,44,45,46,47,48,49,50,51,52,56,57,58,59,60,61,65,66,67,68,69,70,71,72,73,74,75,76,80,81,82,83,84,85,89,90,91,92,93,94,95,96,97,98,99,100,104,105,106,107,108,109,113,114,115,116,117,118,119,120,121,122,123,124,128,129,130,131,132,133,137,138,139,140,141,142,143,144,145,146,147,148,152,153,154,155,156,157,161,162,163,164,165,166,167,168,169,170,171,172,176,177,178,179,180,181,185,186,187,188,189,190,191,192,193,194,195,196,200,201,202,203,204,205,209,210,211,212,213,214,215,216,217,218,219,220,224,225,226,227,228,229,233,234,235,236,237,238,239,240,241,242,243,244,248,249,250,251,252,253,257,258,259,260,261,262,263,264,265,266,267,268,272,273,274) have mixed types. Specify dtype option on import or set low_memory=False.
  matches_train_data = pd.read_csv('../data/raw/matches.csv')


,teamOneWon,teamTwoWon,map1Team1SideBothPlayer1PlusMinus,map1Team1SideCounterTerroristPlayer1PlusMinus,map1Team1SideTerroristPlayer1PlusMinus,map1Team1SideBothPlayer1Adr,map1Team1SideCounterTerroristPlayer1Adr,map1Team1SideTerroristPlayer1Adr,map1Team1SideBothPlayer1Kast,map1Team1SideCounterTerroristPlayer1Kast,...,map1Team2SideTerroristPlayer5PlusMinus,map1Team2SideBothPlayer5Adr,map1Team2SideCounterTerroristPlayer5Adr,map1Team2SideTerroristPlayer5Adr,map1Team2SideBothPlayer5Kast,map1Team2SideCounterTerroristPlayer5Kast,map1Team2SideTerroristPlayer5Kast,map1Team2SideBothPlayer5Rating,map1Team2SideCounterTerroristPlayer5Rating,map1Team2SideTerroristPlayer5Rating
21,False,True,4,2,1,0.79,1.20,0.74,0.77,0.92,...,-5,0.46,0.27,0.37,0.68,0.70,0.67,0.58,0.52,0.56
22,False,True,13,14,0,1.11,1.45,0.88,0.95,1.00,...,-10,0.45,0.49,0.29,0.58,0.57,0.42,0.38,0.47,0.26
23,False,True,21,11,10,1.36,1.41,1.34,0.88,1.00,...,-5,0.56,0.65,0.36,0.59,0.58,0.60,0.45,0.54,0.25
24,True,False,0,3,-3,0.68,0.80,0.48,0.74,0.75,...,-4,0.44,0.56,0.33,0.63,0.71,0.58,0.84,0.9,0.71
25,True,False,3,-3,6,0.94,0.88,1.00,0.83,0.83,...,-2,0.45,0.57,0.45,0.75,0.58,0.67,0.78,0.66,0.81


(448, 122)

Data saved to ../data/processed/full_matches_train_data.csv
